In [47]:
# https://dacon.io/competitions/official/229255/codeshare/511?page=1&dtype=recent&ptype=pub

In [48]:
# base
import os
import shutil
import random
import numpy as np
import time
from collections import Counter
from sklearn.metrics import mean_squared_error

# data preprocessing
import pandas as pd
pd.set_option('max_columns', 130, 'max_rows', 30)

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

# ignore warining
import warnings
warnings.filterwarnings('ignore')


import folium # 지도 관련 시각화
from folium.plugins import MarkerCluster #지도 관련 시각화
import geopy.distance #거리 계산해주는 패키지 사용


# save
from sklearn.externals import joblib 
import pickle

import geopy.distance

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.externals import joblib 
import pickle

import random
random.seed(42)

import lightgbm as lgb
from keras import metrics
from sklearn.metrics import r2_score

In [49]:
basepath='/content/drive/MyDrive/colab전용/data13/'
train = pd.read_csv(basepath+"train.csv")
test = pd.read_csv(basepath+"test.csv")
bts = pd.read_csv(basepath+"bus_bts.csv")

train.shape, test.shape, bts.shape

((415423, 21), (228170, 20), (2409414, 13))

In [50]:
# 생략.
weather_data = pd.read_csv(basepath+'weather.csv',encoding='cp949')

In [51]:
# 요일

train['date'] = pd.to_datetime(train['date'])
train['weekday'] = train['date'].dt.weekday

test['date'] = pd.to_datetime(test['date'])
test['weekday'] = test['date'].dt.weekday

train.shape, test.shape

((415423, 22), (228170, 21))

In [52]:
# 노선과 station_code 

train['bus_route_id'] = train['bus_route_id'].astype(str)
train['station_code'] = train['station_code'].astype(str)
train['route_station'] = train['bus_route_id'] + ',' + train['station_code']

test['bus_route_id'] = test['bus_route_id'].astype(str)
test['station_code'] = test['station_code'].astype(str)
test['route_station'] = test['bus_route_id'] + ',' + test['station_code']

train.shape, test.shape

((415423, 23), (228170, 22))

In [53]:
# 요일과 노선

train['bus_route_id_weekday'] = train['bus_route_id'].astype(str) + ',' + train['weekday'].astype(str) 
test['bus_route_id_weekday'] = test['bus_route_id'].astype(str) + ',' + test['weekday'].astype(str) 

train.shape, test.shape

((415423, 24), (228170, 23))

In [54]:
# 요일과 station_code 

train['station_code_weekday'] = train['station_code'].astype(str) + ',' + train['weekday'].astype(str)
test['station_code_weekday'] = test['station_code'].astype(str) + ',' + test['weekday'].astype(str)

train.shape, test.shape

((415423, 25), (228170, 24))

In [55]:
# 요일과 route_station 
# 섞은것의 또 섞은것. 

train['route_station_weekday'] = train['route_station'].astype(str) + ',' + train['weekday'].astype(str) 
test['route_station_weekday'] = test['route_station'].astype(str) + ',' + test['weekday'].astype(str)

train.shape, test.shape

((415423, 26), (228170, 25))

In [56]:
# 시간대만 추출.

bts['on_time']  = bts['geton_time'].apply(lambda x : x[:2])

bts.iloc[bts.query('on_time == "06"').index,13] = '6~7_ride'
bts.iloc[bts.query('on_time == "07"').index,13] = '7~8_ride'
bts.iloc[bts.query('on_time == "08"').index,13] = '8~9_ride'
bts.iloc[bts.query('on_time == "09"').index,13] = '9~10_ride'
bts.iloc[bts.query('on_time == "10"').index,13] = '10~11_ride'
bts.iloc[bts.query('on_time == "11"').index,13] = '11~12_ride'

bts.shape

(2409414, 14)

In [57]:
# 시간대 2시간대로 다운샘플링.

train['68a']=train['6~7_ride']+train['7~8_ride'] 
train['810a']=train['8~9_ride']+train['9~10_ride']
train['1012a']=train['10~11_ride']+train['11~12_ride']

train['68b']=train['6~7_takeoff']+train['7~8_takeoff'] 
train['810b']=train['8~9_takeoff']+train['9~10_takeoff']
train['1012b']=train['10~11_takeoff']+train['11~12_takeoff']

test['68a']=test['6~7_ride']+test['7~8_ride']
test['810a']=test['8~9_ride']+test['9~10_ride']
test['1012a']=test['10~11_ride']+test['11~12_ride']

test['68b']=test['6~7_takeoff']+test['7~8_takeoff']
test['810b']=test['8~9_takeoff']+test['9~10_takeoff']
test['1012b']=test['10~11_takeoff']+test['11~12_takeoff']

train.shape, test.shape

((415423, 32), (228170, 31))

In [58]:
def id_statistic(ID, col1, col2) :
    
    # mean, sum
    rs_mean = train.groupby([ID])['18~20_ride'].agg([(col1, 'mean')]).reset_index()
    rs_sum = train.groupby([ID])['18~20_ride'].agg([(col2, 'sum')]).reset_index()
    rs_mean_sum = pd.merge(rs_mean, rs_sum, on=ID)

    # merge
    tr = pd.merge(train, rs_mean_sum, how='left', on=ID)
    te = pd.merge(test, rs_mean_sum, how='left', on=ID)

    # na -&gt; mean
    te[col1] = te[col1].fillna(rs_mean.mean())
    te[col1] = te[col1].fillna(rs_sum.mean())
    
    return tr, te

In [59]:
train, test = id_statistic('route_station', '1820_rs_mean', '1820_rs_sum')
train, test = id_statistic('bus_route_id', '1820_r_mean', '1820_r_sum')
train, test = id_statistic('station_code', '1820_s_mean', '1820_s_sum')
train, test = id_statistic('weekday', '1820_w_mean', '1820_w_sum')

train.shape, test.shape

((415423, 40), (228170, 39))

In [60]:
def mean_statistics() :

    f = train.groupby(['bus_route_id_weekday'])['18~20_ride'].agg([('mean_bus_weekday_ride','mean')]).reset_index()
    tr = pd.merge(train, f, how='left', on='bus_route_id_weekday')
    te = pd.merge(test, f, how='left', on='bus_route_id_weekday').fillna(f['mean_bus_weekday_ride'].mean())
    
    f = train.groupby(['station_code_weekday'])['18~20_ride'].agg([('mean_station_weekday_ride','mean')]).reset_index()
    tr = pd.merge(tr, f, how='left', on='station_code_weekday')
    te = pd.merge(te, f, how='left', on='station_code_weekday').fillna(f['mean_station_weekday_ride'].mean())
    
    f = train.groupby(['route_station_weekday'])['18~20_ride'].agg([('mean_route_station_weekday_ride','mean')]).reset_index()
    tr = pd.merge(tr, f, how='left', on='route_station_weekday')
    te = pd.merge(te, f, how='left', on='route_station_weekday').fillna(f['mean_route_station_weekday_ride'].mean())
    
    return tr, te

In [61]:
train, test = mean_statistics()

train.shape, test.shape

((415423, 43), (228170, 42))

In [62]:
# 혼잡도

def congestion() :
    df = train.groupby(['bus_route_id'])['18~20_ride'].agg([('passenger', 'sum')])
    df = df.sort_values(by='passenger', ascending=False).reset_index()
    
    def f(x):
        if x > 10000:
            return 7

        elif x > 5000:
            return 6

        elif x > 2000:
            return 5

        elif x > 700:
            return 4

        elif x > 200:
            return 3

        elif x > 50:
            return 2

        else:
            return 1
    
    df['congestion']=df['passenger'].apply(f)
    df = df[['bus_route_id','congestion']]
    
    tr = pd.merge(train, df, how='left', on='bus_route_id')
    te = pd.merge(test, df, how='left', on='bus_route_id')
    
    # 결측치는 데이터 프레임 df의 'congestion'의 중간값인 '4'으로 대체
    te = te.fillna(4)
    
    return tr, te

train, test = congestion()

train.shape, test.shape

((415423, 44), (228170, 43))

In [63]:
# location
train['location'] = train['latitude'].astype(str) + ',' + train['longitude'].astype(str)
test['location'] = test['latitude'].astype(str) + ',' + test['longitude'].astype(str)

train.shape, test.shape

((415423, 45), (228170, 44))

In [64]:
# make cue column 무슨뜻인지는 잘 모르겠다. merge를 위한 키라는데. 
train['cue']=0
test['cue']=1

train.shape, test.shape

((415423, 46), (228170, 45))

In [65]:
def morning() :
    
    # merge
    data = pd.concat([train, test])
    
    a = data.groupby(['route_station'])['1012a'].agg({'sum', 'mean'}).reset_index()
    a.columns = ['route_station', '1012a_sum','1012a_mean']

    b = data.groupby(['route_station'])['1012b'].agg({'sum', 'mean'}).reset_index()
    b.columns = ['route_station', '1012b_sum','1012b_mean']
    b = b[['1012b_sum','1012b_mean']]

    c = data.groupby(['route_station'])['10~11_ride'].agg({'sum', 'mean'}).reset_index()
    c.columns = ['route_station', '10~11_ride_sum','10~11_ride_mean']
    c = c[['10~11_ride_sum','10~11_ride_mean']]

    d = data.groupby(['route_station'])['10~11_takeoff'].agg({'sum', 'mean'}).reset_index()
    d.columns = ['route_station', '10~11_takeoff_sum','10~11_takeoff_mean']
    d = d[['10~11_takeoff_sum','10~11_takeoff_mean']]

    e = data.groupby(['route_station'])['11~12_ride'].agg({'sum', 'mean'}).reset_index()
    e.columns = ['route_station', '11~12_ride_sum','11~12_ride_mean']
    e = e[['11~12_ride_sum','11~12_ride_mean']]

    f = data.groupby(['route_station'])['11~12_takeoff'].agg({'sum', 'mean'}).reset_index()
    f.columns = ['route_station', '11~12_takeoff_sum','11~12_takeoff_mean']
    f = f[['11~12_takeoff_sum','11~12_takeoff_mean']]

    g = data.groupby(['route_station'])['1820_r_mean'].agg({'sum', 'mean'}).reset_index()
    g.columns = ['route_station', '1820_r_mean_sum','1820_r_mean_mean']
    g = g[['1820_r_mean_sum','1820_r_mean_mean']]

    h = data.groupby(['route_station'])['1820_r_sum'].agg({'sum', 'mean'}).reset_index()
    h.columns = ['route_station', '1820_r_sum_sum','1820_r_sum_mean']
    h = h[['1820_r_sum_sum','1820_r_sum_mean']]

    i = data.groupby(['route_station'])['1820_rs_mean'].agg({'sum', 'mean'}).reset_index()
    i.columns = ['route_station', '1820_rs_mean_sum','1820_rs_mean_mean']
    i = i[['1820_rs_mean_sum','1820_rs_mean_mean']]

    j = data.groupby(['route_station'])['1820_rs_sum'].agg({'sum', 'mean'}).reset_index()
    j.columns = ['route_station', '1820_rs_sum_sum','1820_rs_sum_mean']
    j = j[['1820_rs_sum_sum','1820_rs_sum_mean']]

    k = data.groupby(['route_station'])['1820_s_mean'].agg({'sum', 'mean'}).reset_index()
    k.columns = ['route_station', '1820_s_mean_sum','1820_s_mean_mean']
    k = k[['1820_s_mean_sum','1820_s_mean_mean']]

    l = data.groupby(['route_station'])['1820_s_sum'].agg({'sum', 'mean'}).reset_index()
    l.columns = ['route_station', '1820_s_sum_sum','1820_s_sum_mean']
    l = l[['1820_s_sum_sum','1820_s_sum_mean']]

    m = data.groupby(['route_station'])['1820_w_mean'].agg({'sum', 'mean'}).reset_index()
    m.columns = ['route_station', '1820_w_mean_sum','1820_w_mean_mean']
    m = m[['1820_w_mean_sum','1820_w_mean_mean']]

    n = data.groupby(['route_station'])['1820_w_sum'].agg({'sum', 'mean'}).reset_index()
    n.columns = ['route_station', '1820_w_sum_sum','1820_w_sum_mean']
    n = n[['1820_w_sum_sum','1820_w_sum_mean']]

    o = data.groupby(['route_station'])['68a'].agg({'sum', 'mean'}).reset_index()
    o.columns = ['route_station', '68a_sum','68a_mean']
    o = o[['68a_sum','68a_mean']]

    p = data.groupby(['route_station'])['68b'].agg({'sum', 'mean'}).reset_index()
    p.columns = ['route_station', '68b_sum','68b_mean']
    p = p[['68b_sum','68b_mean']]

    q = data.groupby(['route_station'])['6~7_ride'].agg({'sum', 'mean'}).reset_index()
    q.columns = ['route_station', '6~7_ride_sum','6~7_ride_mean']
    q = q[['6~7_ride_sum','6~7_ride_mean']]

    r = data.groupby(['route_station'])['6~7_takeoff'].agg({'sum', 'mean'}).reset_index()
    r.columns = ['route_station', '6~7_takeoff_sum','6~7_takeoff_mean']
    r = r[['6~7_takeoff_sum','6~7_takeoff_mean']]

    s = data.groupby(['route_station'])['7~8_ride'].agg({'sum', 'mean'}).reset_index()
    s.columns = ['route_station', '7~8_ride_sum','7~8_ride_mean']
    s = s[['7~8_ride_sum','7~8_ride_mean']]

    t = data.groupby(['route_station'])['7~8_takeoff'].agg({'sum', 'mean'}).reset_index()
    t.columns = ['route_station', '7~8_takeoff_sum','7~8_takeoff_mean']
    t = t[['7~8_takeoff_sum','7~8_takeoff_mean']]

    u = data.groupby(['route_station'])['810a'].agg({'sum', 'mean'}).reset_index()
    u.columns = ['route_station', '810a_sum','810a_mean']
    u = u[['810a_sum','810a_mean']]

    v = data.groupby(['route_station'])['810b'].agg({'sum', 'mean'}).reset_index()
    v.columns = ['route_station', '810b_sum','810b_mean']
    v = v[['810b_sum','810b_mean']]

    w = data.groupby(['route_station'])['8~9_ride'].agg({'sum', 'mean'}).reset_index()
    w.columns = ['route_station', '8~9_ride_sum','8~9_ride_mean']
    w = w[['8~9_ride_sum','8~9_ride_mean']]

    x = data.groupby(['route_station'])['8~9_takeoff'].agg({'sum', 'mean'}).reset_index()
    x.columns = ['route_station', '8~9_takeoff_sum','8~9_takeoff_mean']
    x = x[['8~9_takeoff_sum','8~9_takeoff_mean']]

    y = data.groupby(['route_station'])['9~10_ride'].agg({'sum', 'mean'}).reset_index()
    y.columns = ['route_station', '9~10_ride_sum','9~10_ride_mean']
    y = y[['9~10_ride_sum','9~10_ride_mean']]

    z = data.groupby(['route_station'])['9~10_takeoff'].agg({'sum', 'mean'}).reset_index()
    z.columns = ['route_station', '9~10_takeoff_sum','9~10_takeoff_mean']
    z = z[['9~10_takeoff_sum','9~10_takeoff_mean']]
    
    df = pd.concat([a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z],axis=1)
    df = pd.merge(data, df, how='left', on='route_station')
    
    return df

data = morning()

data.shape

(643593, 98)

In [66]:
# 배차간격

train['bus_route_id'] = train['bus_route_id'].astype(np.int64)
test['bus_route_id'] = test['bus_route_id'].astype(np.int64)

bts['geton_time2'] = pd.to_datetime(bts['geton_time'])

In [67]:
# sample=bts.groupby(['geton_date','geton_time2','geton_station_code','bus_route_id'])['user_count'].\
# agg([('탑승객_수','sum')]).reset_index().\
# sort_values(by=['geton_date','geton_station_code','bus_route_id','geton_time2'], ascending=True).reset_index(drop=True)

# sample['bus_route_id_diff']=sample['bus_route_id'].diff(1)
# sample['geton_station_code_diff']=sample['geton_station_code'].diff(1)

# sample['geton_time2_diff']=sample['geton_time2'].diff(1)

# for i in range(1,sample.shape[0]-1):
#     if sample.iloc[i].geton_date==sample.iloc[i+1].geton_date:
#         if 

In [68]:
# f = bts.groupby(['geton_date','geton_time2','geton_station_code','bus_route_id'])['user_count'].\
# agg([('탑승객_수','sum')]).reset_index().\
# sort_values(by=['geton_date','geton_station_code','bus_route_id','geton_time2'], ascending=True).reset_index()

# f['index'] = list(range(0,len(f)))

# time = []

# for i in range(0,len(f)-1):

#     if ((f.iloc[i].geton_date == f.iloc[i+1].geton_date) &\
#         (f.iloc[i].geton_station_code == f.iloc[i+1].geton_station_code) &\
#         (f.iloc[i].bus_route_id == f.iloc[i+1].bus_route_id)):

#         time.append(f.iloc[i+1].geton_time2 - f.iloc[i].geton_time2)

#     else:
#         time.append(0)

# time.insert(0, '0')

In [69]:
# def get_sec(time_str):

#     h, m, s = time_str.split(':')

#     return int(h) * 3600 + int(m) * 60 + int(s)

# def bus_interval() :

#     f['time'] = time
#     f['time2'] = f['time'].astype(str).str[7:]


#     interval = f.copy()
#     interval['time2'] = interval['time2'].astype(str).replace('','00:00:00')
#     interval['bus_route_id'] = interval['bus_route_id'].astype(object)

#     time4 = []

#     for i in interval['time2'] :

#         time4.append(get_sec(i))

#     interval['time4'] = time4
#     interval['time4'] = (interval['time4'] / 60).astype(int)

#     interval = interval[interval['time4'] > 3] # 간격이 3분보다 작은 것 제외 
#     interval = interval[interval['time4'] < 180] # 간격이 3시간보다 큰 것 제외

#     interval = interval.groupby('bus_route_id')['time4'].agg([('bus_interval', 'mean')]).reset_index()
#     interval['bus_interval'] = interval['bus_interval'].astype(int)

#     # 나중에 시간을 절약하기 위해 csv 파일로 저장
#     interval.to_csv(basepath+'bus_interval_final.csv', index = False)

#     print('success.. !')
    
# bus_interval()

In [73]:
def weather() :

    weather_data = pd.read_csv(basepath+'weather.csv', encoding = 'cp949')
    weather_data['id'] = range(0,46)

    a = pd.DataFrame(data.date.unique(), columns=['date']) ; a['id'] = range(0,46)
    weather_data = pd.merge(a, weather_data)
    weather_data = weather_data[['date','현재일기_10','체감온도_10','일강수_10']]
    weather_data = weather_data.replace(' ', 0)
    df = pd.merge(data, weather_data, on='date')
    # label_encoder
    labelencoder = LabelEncoder()
    df_encode = df[['현재일기_10']]
    df_encoded = df_encode.apply(labelencoder.fit_transform)
    df['현재일기_10']=df_encoded['현재일기_10']
    # object-&gt;float 변수변환
    df['현재일기_10'] = df['현재일기_10'].astype(float)
    df['체감온도_10'] = df['체감온도_10'].astype(float)
    df['일강수_10'] = df['일강수_10'].astype(float)
    return df

data = weather()

data.shape

(643593, 101)

In [75]:
data['date'] = pd.to_datetime(data['date'])
data['weekday'] = data['date'].dt.weekday
data = pd.get_dummies(data,columns=['weekday'])
data['weekday'] = data['date'].dt.weekday
data.shape

(643593, 108)

In [76]:
data['in_out'].value_counts()
data['in_out'] = data['in_out'].map({'시내':0,'시외':1})

data.shape

(643593, 108)

In [77]:
# # 많은 시간 소요.
# coords_jejusi = (33.500770, 126.522761) #제주시의 위도 경도
# data['dis_jejusi'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejusi).km for i in range(len(data))]

# coords_jejusicheong1 = (33.49892, 126.53035) #제주시청(광양방면)의 위도 경도
# coords_jejuairport = (33.50661, 126.49345) #제주국제공항(구제주방면)의 위도 경도
# coords_hallahosp = (33.48963, 126.486) #한라병원의 위도 경도
# coords_rotary = (33.49143, 126.49678) # 제주도청신제주로터리의 위도 경도
# coords_jejucenterhigh = (33.48902, 126.5392) #제주중앙여자고등학교의 위도 경도
# coords_jejumarket = (33.51315, 126.52706) #동문시장의 위도 경도
# coords_jejusclass = (33.47626, 126.48141) #제주고등학교/중흥S클래스의 위도 경도
# coords_centerroad = (33.51073, 126.5239) #중앙로(국민은행)의 위도 경도
# coords_fiveway = (33.48667, 126.48092) # 노형오거리의 위도 경도
# coords_law = (33.49363, 126.53476) # 제주지방법원(광양방면)의 위도 경도

# data['dis_jejusicheong1'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejusicheong1).km for i in range(len(data))]
# data['dis_jejuairport'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejuairport).km for i in range(len(data))]
# data['dis_hallahosp'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_hallahosp).km for i in range(len(data))]
# data['dis_rotary'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_rotary).km for i in range(len(data))]
# data['dis_jejucenterhigh'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejucenterhigh).km for i in range(len(data))]
# data['dis_jejumarket'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejumarket).km for i in range(len(data))]
# data['dis_jejusclass'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejusclass).km for i in range(len(data))]
# data['dis_centerroad'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_centerroad).km for i in range(len(data))]
# data['dis_fiveway'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_fiveway).km for i in range(len(data))]
# data['dis_law'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_law).km for i in range(len(data))]

# data.shape

In [78]:
data['ride_sum'] = data['6~7_ride'] + data['7~8_ride'] + data['8~9_ride'] + data['9~10_ride'] + data['10~11_ride'] + data['11~12_ride'] 
data['takeoff_sum'] = data['6~7_takeoff'] + data['7~8_takeoff'] + data['8~9_takeoff'] + data['9~10_takeoff'] + data['10~11_takeoff'] + data['11~12_takeoff'] 

data.shape

(643593, 110)

In [79]:
f = data.groupby('date')['6~7_ride'].agg([('6~7_all_ride_number', 'sum')]).reset_index()
data = pd.merge(data, f, how='left')

f = data.groupby('date')['7~8_ride'].agg([('7~8_all_ride_number', 'sum')]).reset_index()
data = pd.merge(data, f, how='left')

f = data.groupby('date')['8~9_ride'].agg([('8~9_all_ride_number', 'sum')]).reset_index()
data = pd.merge(data, f, how='left')

f = data.groupby('date')['9~10_ride'].agg([('9~10_all_ride_number', 'sum')]).reset_index()
data = pd.merge(data, f, how='left')

f = data.groupby('date')['10~11_ride'].agg([('10~11_all_ride_number', 'sum')]).reset_index()
data = pd.merge(data, f, how='left')

data.shape

(643593, 115)

In [80]:
def h(x):
    if x ==5:
        return 1
    elif x==6:
        return 1
    else:
        return 0

data['weekend'] = data['weekday'].map(h)

data.shape

(643593, 116)

In [81]:
def g(x):
    if x in ['2019-09-12','2019-09-13','2019-09-14','2019-10-03','2019-10-09']:
        return 1
    else:
        return 0

data['holiday'] = data['date'].apply(g) 

data.shape

(643593, 117)

In [82]:
def week_mean() :

    df = data.reset_index(drop=True)
    df.groupby('weekday')['18~20_ride'].mean()
    df['weekdaymean']= 1

    index0 = df.query('weekday==0').index
    index1 = df.query('weekday==1').index
    index2 = df.query('weekday==2').index
    index3 = df.query('weekday==3').index
    index4 = df.query('weekday==4').index
    index5 = df.query('weekday==5').index
    index6 = df.query('weekday==6').index

    df.iloc[index0,-1] = 1.343710
    df.iloc[index1,-1] = 1.375319
    df.iloc[index2,-1] = 1.430856
    df.iloc[index3,-1] = 1.256710
    df.iloc[index4,-1] = 1.067439
    df.iloc[index5,-1] = 1.062123
    df.iloc[index6,-1] = 1.034282

    return df

data = week_mean()

data.shape

(643593, 118)

In [83]:
data['in_out_mean'] = 1
inindex = data.query('in_out == "시내"').index
outindex = data.query('in_out == "시외"').index

data.iloc[inindex,-1] = 1.228499
data.iloc[outindex,-1] = 2.044345
data['congestion'] = data['congestion'].astype('int64')

data.shape

(643593, 119)

In [84]:
def category_people() :
    bts['bus_route_id'] = bts['bus_route_id'].astype(str)

    f = bts.groupby(['bus_route_id','user_category'])['user_count'].agg([('승객수', 'sum')]).reset_index()

    g = pd.pivot_table(f, values='승객수', index='bus_route_id', columns='user_category',fill_value=0).reset_index()
    g.columns = ['bus_route_id', 'adult','kids','teen','elder','jang','jang2','ugong','ugong2']
    g = g[['bus_route_id', 'adult','kids','teen','elder']]

    # merge
    df = pd.merge(data, g, how='left', on='bus_route_id')

    # na preprocessing -&gt; mean value
    df['adult'] = df['adult'].fillna(2363.077778)
    df['kids'] = df['kids'].fillna(60.426984)
    df['teen'] = df['teen'].fillna(448.277778)
    df['elder'] = df['elder'].fillna(751.309524)
                 
    return df

In [86]:
data['bus_route_id'] = data['bus_route_id'].astype(str)

In [87]:
data = category_people()

data.shape

(643593, 123)